# Fuzzy Language in Data Science

In [1]:
import pandas as pd
import datetime as dt

In [3]:
data = pd.read_csv("rfm_xmas19.csv", parse_dates=["trans_date"])
group_by_customer = data.groupby("customer_id")
last_transaction = group_by_customer["trans_date"].max()
best_churn = pd.DataFrame(last_transaction)

cutoff_day = dt.datetime(2019, 10, 16)

best_churn["churned"] = best_churn["trans_date"].apply(
    lambda date: 1 if date < cutoff_day else 0
)

In [4]:
best_churn["nr_of_transactions"] = group_by_customer.size()

best_churn['amount_spent'] = group_by_customer.sum()

In [5]:
best_churn = best_churn.drop('trans_date',axis=1)

In [6]:
best_churn['scaled_tran'] = (best_churn['nr_of_transactions']-best_churn['nr_of_transactions'].min())/(best_churn['nr_of_transactions'].max()-best_churn['nr_of_transactions'].min())

best_churn['scaled_amount'] = (best_churn['amount_spent']-best_churn['amount_spent'].min())/(best_churn['amount_spent'].max()-best_churn['amount_spent'].min())

best_churn['score'] = best_churn['scaled_amount'] * 0.5 + 0.5 * best_churn['scaled_tran']

best_churn['score'] = 100 * best_churn['score']

best_churn = best_churn.sort_values('score',ascending=False)

In [7]:
best_churn.head()

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
CS4424,1,39,2933,1.000000,1.000000,100.000000
CS4320,1,38,2647,0.971429,0.897270,93.434934
CS3799,1,36,2513,0.914286,0.849138,88.171182
CS5109,1,35,2506,0.885714,0.846624,86.616892
CS3805,1,35,2453,0.885714,0.827586,85.665025


In [8]:
coupon = (data['tran_amount'].mean()) * 0.3

nr_of_customers = 1000 / coupon

In [9]:
print(coupon, nr_of_customers, sep="\n")

19.4975736
51.28843314123969


In [10]:
top_50_churned = best_churn.loc[best_churn['churned']==1].head(50)

top_50_churned.to_csv('best_customers.csv')